# Listing Physical Devices

In [1]:
import tensorflow as tf
tf.config.list_physical_devices(
    device_type=None
)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Importing the Modules

In [3]:
# Used in Tensorflow Model
tf.get_logger().setLevel('ERROR')

import numpy as np
import tflearn
import random

# used to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

tf.get_logger().setLevel('ERROR')

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Loading the Data

In [4]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

# Converting the data to words, classes, documents and ignore words

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        
        # add to our words list
        words.extend(w)
        
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Stemming, case folding and Removing Duplicates

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


# Creating a bag of words for training the model

In [7]:
training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffling Randomly and Converting into Numpy Array for Faster Processing

In [8]:
random.shuffle(training)
training = np.array(training)

# Creating Train and Test Lists
train_x = list(training[:,0])
train_y = list(training[:,1])

# Building Neural Network for Out Chatbot to be Contextual
# Resetting graph data
tf.compat.v1.reset_default_graph()

# Creating the model

In [9]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [10]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Training the model

In [11]:
model.fit(train_x, train_y, n_epoch=1000, batch_size=32, show_metric=True)

# Saving the Model
model.save('model.tflearn')

---------------------------------
Run id: FGG452
Log directory: tflearn_logs/
---------------------------------
Training samples: 27
Validation samples: 0
--
Training Step: 1  | time: 0.825s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 27/27
--
Training Step: 2  | total loss: 1.97750
| Adam | epoch: 002 | loss: 1.97750 - acc: 0.1667 -- iter: 27/27
--
Training Step: 3  | total loss: 2.15715 | time: 0.009s
| Adam | epoch: 003 | loss: 2.15715 - acc: 0.1818 -- iter: 27/27
--
Training Step: 4  | total loss: 2.18696 | time: 0.007s
| Adam | epoch: 004 | loss: 2.18696 - acc: 0.1843 -- iter: 27/27
--
Training Step: 5  | total loss: 2.19371 | time: 0.003s
| Adam | epoch: 005 | loss: 2.19371 - acc: 0.1849 -- iter: 27/27
--
Training Step: 6  | total loss: 2.19552 | time: 0.003s
| Adam | epoch: 006 | loss: 2.19552 - acc: 0.1851 -- iter: 27/27
--
Training Step: 7  | total loss: 2.19602 | time: 0.004s
| Adam | epoch: 007 | loss: 2.19602 - acc: 0.1851 -- iter: 27/27
--
Training Step: 8  

Training Step: 60  | total loss: 2.17568 | time: 0.004s
| Adam | epoch: 060 | loss: 2.17568 - acc: 0.1852 -- iter: 27/27
--
Training Step: 61  | total loss: 2.17303
| Adam | epoch: 061 | loss: 2.17303 - acc: 0.1852 -- iter: 27/27
--
Training Step: 62  | total loss: 2.17055 | time: 0.008s
| Adam | epoch: 062 | loss: 2.17055 - acc: 0.1852 -- iter: 27/27
--
Training Step: 63  | total loss: 2.16821 | time: 0.003s
| Adam | epoch: 063 | loss: 2.16821 - acc: 0.1852 -- iter: 27/27
--
Training Step: 64  | total loss: 2.16594 | time: 0.003s
| Adam | epoch: 064 | loss: 2.16594 - acc: 0.1852 -- iter: 27/27
--
Training Step: 65  | total loss: 2.16373 | time: 0.001s
| Adam | epoch: 065 | loss: 2.16373 - acc: 0.1852 -- iter: 27/27
--
Training Step: 66  | total loss: 2.16595 | time: 0.007s
| Adam | epoch: 066 | loss: 2.16595 - acc: 0.1852 -- iter: 27/27
--
Training Step: 67  | total loss: 2.16328 | time: 0.003s
| Adam | epoch: 067 | loss: 2.16328 - acc: 0.1896 -- iter: 27/27
--
Training Step: 68  | to

Training Step: 121  | total loss: 1.93860 | time: 0.001s
| Adam | epoch: 121 | loss: 1.93860 - acc: 0.3162 -- iter: 27/27
--
Training Step: 122  | total loss: 1.92819 | time: 0.007s
| Adam | epoch: 122 | loss: 1.92819 - acc: 0.3179 -- iter: 27/27
--
Training Step: 123  | total loss: 1.91787 | time: 0.003s
| Adam | epoch: 123 | loss: 1.91787 - acc: 0.3195 -- iter: 27/27
--
Training Step: 124  | total loss: 1.90763 | time: 0.005s
| Adam | epoch: 124 | loss: 1.90763 - acc: 0.3209 -- iter: 27/27
--
Training Step: 125  | total loss: 1.89744
| Adam | epoch: 125 | loss: 1.89744 - acc: 0.3221 -- iter: 27/27
--
Training Step: 126  | total loss: 1.93872 | time: 0.009s
| Adam | epoch: 126 | loss: 1.93872 - acc: 0.3047 -- iter: 27/27
--
Training Step: 127  | total loss: 1.92363 | time: 0.002s
| Adam | epoch: 127 | loss: 1.92363 - acc: 0.3113 -- iter: 27/27
--
Training Step: 128  | total loss: 1.90921 | time: 0.001s
| Adam | epoch: 128 | loss: 1.90921 - acc: 0.3172 -- iter: 27/27
--
Training Step: 

Training Step: 182  | total loss: 1.42037 | time: 0.007s
| Adam | epoch: 182 | loss: 1.42037 - acc: 0.4825 -- iter: 27/27
--
Training Step: 183  | total loss: 1.41190 | time: 0.001s
| Adam | epoch: 183 | loss: 1.41190 - acc: 0.4861 -- iter: 27/27
--
Training Step: 184  | total loss: 1.40351 | time: 0.009s
| Adam | epoch: 184 | loss: 1.40351 - acc: 0.4930 -- iter: 27/27
--
Training Step: 185  | total loss: 1.39519 | time: 0.002s
| Adam | epoch: 185 | loss: 1.39519 - acc: 0.4993 -- iter: 27/27
--
Training Step: 186  | total loss: 1.38696 | time: 0.007s
| Adam | epoch: 186 | loss: 1.38696 - acc: 0.5049 -- iter: 27/27
--
Training Step: 187  | total loss: 1.37880
| Adam | epoch: 187 | loss: 1.37880 - acc: 0.5100 -- iter: 27/27
--
Training Step: 188  | total loss: 1.37071 | time: 0.006s
| Adam | epoch: 188 | loss: 1.37071 - acc: 0.5145 -- iter: 27/27
--
Training Step: 189  | total loss: 1.36269
| Adam | epoch: 189 | loss: 1.36269 - acc: 0.5186 -- iter: 27/27
--
Training Step: 190  | total lo

Training Step: 243  | total loss: 1.20992 | time: 0.003s
| Adam | epoch: 243 | loss: 1.20992 - acc: 0.7231 -- iter: 27/27
--
Training Step: 244  | total loss: 1.19189 | time: 0.004s
| Adam | epoch: 244 | loss: 1.19189 - acc: 0.7323 -- iter: 27/27
--
Training Step: 245  | total loss: 1.17520 | time: 0.005s
| Adam | epoch: 245 | loss: 1.17520 - acc: 0.7405 -- iter: 27/27
--
Training Step: 246  | total loss: 1.15970 | time: 0.003s
| Adam | epoch: 246 | loss: 1.15970 - acc: 0.7479 -- iter: 27/27
--
Training Step: 247  | total loss: 1.14528 | time: 0.003s
| Adam | epoch: 247 | loss: 1.14528 - acc: 0.7546 -- iter: 27/27
--
Training Step: 248  | total loss: 1.31946 | time: 0.006s
| Adam | epoch: 248 | loss: 1.31946 - acc: 0.6940 -- iter: 27/27
--
Training Step: 249  | total loss: 1.28824 | time: 0.003s
| Adam | epoch: 249 | loss: 1.28824 - acc: 0.7061 -- iter: 27/27
--
Training Step: 250  | total loss: 1.25979 | time: 0.008s
| Adam | epoch: 250 | loss: 1.25979 - acc: 0.7169 -- iter: 27/27
--


...

Training Step: 729  | total loss: 0.64319
| Adam | epoch: 729 | loss: 0.64319 - acc: 0.9193 -- iter: 27/27
--
Training Step: 730  | total loss: 1.03908 | time: 0.008s
| Adam | epoch: 730 | loss: 1.03908 - acc: 0.8385 -- iter: 27/27
--
Training Step: 731  | total loss: 0.96468 | time: 0.007s
| Adam | epoch: 731 | loss: 0.96468 - acc: 0.8546 -- iter: 27/27
--
Training Step: 732  | total loss: 0.89775
| Adam | epoch: 732 | loss: 0.89775 - acc: 0.8692 -- iter: 27/27
--
Training Step: 733  | total loss: 0.83752 | time: 0.009s
| Adam | epoch: 733 | loss: 0.83752 - acc: 0.8823 -- iter: 27/27
--
Training Step: 734  | total loss: 0.78331 | time: 0.003s
| Adam | epoch: 734 | loss: 0.78331 - acc: 0.8940 -- iter: 27/27
--
Training Step: 735  | total loss: 0.73448 | time: 0.006s
| Adam | epoch: 735 | loss: 0.73448 - acc: 0.9046 -- iter: 27/27
--
Training Step: 736  | total loss: 0.69050
| Adam | epoch: 736 | loss: 0.69050 - acc: 0.9142 -- iter: 27/27
--
Training Step: 737  | total loss: 0.65087 | t

Training Step: 790  | total loss: 0.97610
| Adam | epoch: 790 | loss: 0.97610 - acc: 0.8621 -- iter: 27/27
--
Training Step: 791  | total loss: 0.90971
| Adam | epoch: 791 | loss: 0.90971 - acc: 0.8759 -- iter: 27/27
--
Training Step: 792  | total loss: 1.25332 | time: 0.000s
| Adam | epoch: 792 | loss: 1.25332 - acc: 0.8031 -- iter: 27/27
--
Training Step: 793  | total loss: 1.15930 | time: 0.007s
| Adam | epoch: 793 | loss: 1.15930 - acc: 0.8228 -- iter: 27/27
--
Training Step: 794  | total loss: 1.07475
| Adam | epoch: 794 | loss: 1.07475 - acc: 0.8405 -- iter: 27/27
--
Training Step: 795  | total loss: 0.99871 | time: 0.008s
| Adam | epoch: 795 | loss: 0.99871 - acc: 0.8565 -- iter: 27/27
--
Training Step: 796  | total loss: 0.93029
| Adam | epoch: 796 | loss: 0.93029 - acc: 0.8708 -- iter: 27/27
--
Training Step: 797  | total loss: 0.86872 | time: 0.000s
| Adam | epoch: 797 | loss: 0.86872 - acc: 0.8837 -- iter: 27/27
--
Training Step: 798  | total loss: 0.81328
| Adam | epoch: 79

Training Step: 851  | total loss: 0.69033 | time: 0.003s
| Adam | epoch: 851 | loss: 0.69033 - acc: 0.9055 -- iter: 27/27
--
Training Step: 852  | total loss: 0.65071 | time: 0.005s
| Adam | epoch: 852 | loss: 0.65071 - acc: 0.9150 -- iter: 27/27
--
Training Step: 853  | total loss: 0.61498 | time: 0.003s
| Adam | epoch: 853 | loss: 0.61498 - acc: 0.9235 -- iter: 27/27
--
Training Step: 854  | total loss: 0.95564 | time: 0.004s
| Adam | epoch: 854 | loss: 0.95564 - acc: 0.8422 -- iter: 27/27
--
Training Step: 855  | total loss: 0.88935 | time: 0.003s
| Adam | epoch: 855 | loss: 0.88935 - acc: 0.8580 -- iter: 27/27
--
Training Step: 856  | total loss: 1.36757 | time: 0.005s
| Adam | epoch: 856 | loss: 1.36757 - acc: 0.7796 -- iter: 27/27
--
Training Step: 857  | total loss: 1.26021 | time: 0.002s
| Adam | epoch: 857 | loss: 1.26021 - acc: 0.8017 -- iter: 27/27
--
Training Step: 858  | total loss: 1.16370 | time: 0.002s
| Adam | epoch: 858 | loss: 1.16370 - acc: 0.8215 -- iter: 27/27
--


Training Step: 911  | total loss: 0.71380 | time: 0.007s
| Adam | epoch: 911 | loss: 0.71380 - acc: 0.9142 -- iter: 27/27
--
Training Step: 912  | total loss: 1.06896
| Adam | epoch: 912 | loss: 1.06896 - acc: 0.8302 -- iter: 27/27
--
Training Step: 913  | total loss: 0.99256 | time: 0.009s
| Adam | epoch: 913 | loss: 0.99256 - acc: 0.8472 -- iter: 27/27
--
Training Step: 914  | total loss: 0.92382 | time: 0.007s
| Adam | epoch: 914 | loss: 0.92382 - acc: 0.8625 -- iter: 27/27
--
Training Step: 915  | total loss: 0.86195
| Adam | epoch: 915 | loss: 0.86195 - acc: 0.8762 -- iter: 27/27
--
Training Step: 916  | total loss: 0.80625 | time: 0.009s
| Adam | epoch: 916 | loss: 0.80625 - acc: 0.8886 -- iter: 27/27
--
Training Step: 917  | total loss: 0.75609
| Adam | epoch: 917 | loss: 0.75609 - acc: 0.8997 -- iter: 27/27
--
Training Step: 918  | total loss: 0.71089 | time: 0.001s
| Adam | epoch: 918 | loss: 0.71089 - acc: 0.9098 -- iter: 27/27
--
Training Step: 919  | total loss: 0.67014
| A

Training Step: 972  | total loss: 0.93948 | time: 0.003s
| Adam | epoch: 972 | loss: 0.93948 - acc: 0.8699 -- iter: 27/27
--
Training Step: 973  | total loss: 0.87526 | time: 0.003s
| Adam | epoch: 973 | loss: 0.87526 - acc: 0.8829 -- iter: 27/27
--
Training Step: 974  | total loss: 0.81745 | time: 0.004s
| Adam | epoch: 974 | loss: 0.81745 - acc: 0.8946 -- iter: 27/27
--
Training Step: 975  | total loss: 0.76538 | time: 0.007s
| Adam | epoch: 975 | loss: 0.76538 - acc: 0.9052 -- iter: 27/27
--
Training Step: 976  | total loss: 0.71848 | time: 0.004s
| Adam | epoch: 976 | loss: 0.71848 - acc: 0.9146 -- iter: 27/27
--
Training Step: 977  | total loss: 0.67621 | time: 0.005s
| Adam | epoch: 977 | loss: 0.67621 - acc: 0.9232 -- iter: 27/27
--
Training Step: 978  | total loss: 0.63808
| Adam | epoch: 978 | loss: 0.63808 - acc: 0.9309 -- iter: 27/27
--
Training Step: 979  | total loss: 0.60368 | time: 0.007s
| Adam | epoch: 979 | loss: 0.60368 - acc: 0.9378 -- iter: 27/27
--
Training Step: 

# Pickling the model

In [12]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

# Loading the model

In [13]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    

# load our saved model
model.load('./model.tflearn')

In [14]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

# Testing the model

In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- Is anyone there?
Hello, thanks for visiting
You- What hours are you open?
Our hours are 9am-9pm every day
You- What do you rent?
We rent Yamaha, Piaggio and Vespa mopeds
You- Are you cash only?
We accept VISA, Mastercard and AMEX
You- Can we rent a moped?
Are you looking to rent today or later this week?
You- today
For rentals today please call 1-800-MYMOPED
You- That's helpful
My pleasure
You- Goodbye
Have a nice day
